In [3]:
import sys
from itertools import combinations

frequent_set = {}      
support = {}
confident_set = {}

In [4]:
filename = raw_input("Enter the filename: ")


Enter the filename: data.txt


In [5]:
min_sup = input("Enter the minimum support: ")

Enter the minimum support: 2


In [6]:
delimiter = ","
min_confidence = 0.6
data_file = open(filename)

In [7]:
def get_transactions(data_file, delimiter):
    transactions = []
    for line in data_file:
        items = line.strip().split(delimiter)
        #items = list(map(int, items))
        print items
        transactions.append(items)
    return transactions

transactions = get_transactions(data_file, delimiter) 

['Bread', 'Butter', 'Jam']
['Butter', 'Coke']
['Butter', 'Milk']
['Bread', 'Butter', 'Coke']
['Bread', 'Milk']
['Butter', 'Milk']
['Bread', 'Milk']
['Bread', 'Butter', 'Milk', 'Jam']
['Bread', 'Butter', 'Milk']


In [8]:
def get_t_id_list(transactions):
    t_id_list = [[], []]
    transaction_id = 1
    item_index = {}
    i = 0
    for transaction in transactions:
        for item in transaction:
            if item in t_id_list[0]:
                index = item_index[item]
                t_id_list[1][index].append(transaction_id)
            else:
                t_id_list[0].append(item)
                item_index[item] = i

                t_id_list[1].append([transaction_id])
                i += 1

        transaction_id += 1
    return t_id_list

# data structure of t_id_list
# t_id_list = [ [list of itemsets],  [list of transaction ids of the corresponding itemsets]]
# for exaple:
# t_id_list = [ ["Bread", "Milk"], [1, 4, 5], [5, 6]]
# so, the transaction ids which contains Bread are [1, 4, 5]
# and the transaction ids which contains Milk are [5, 6]
t_id_list = get_t_id_list(transactions)    

for i in range(0, len(t_id_list[0])):
    print t_id_list[0][i], t_id_list[1][i]


Bread [1, 4, 5, 7, 8, 9]
Butter [1, 2, 3, 4, 6, 8, 9]
Jam [1, 8]
Coke [2, 4]
Milk [3, 5, 6, 7, 8, 9]


In [9]:
# this function consider only itemsets which are frequent with 
# respect to given minimun support

def get_frequent_sets(t_id_list, min_sup):
    new_frequent_set = [[], []]
    l = len(t_id_list[0])
    for i in range(0, l):
        if len(t_id_list[1][i]) >= min_sup:
            new_frequent_set[0].append(t_id_list[0][i])
            new_frequent_set[1].append(t_id_list[1][i])
            frequent_set[t_id_list[0][i]] = t_id_list[1][i]
    return new_frequent_set

t_id_list = get_frequent_sets(t_id_list, min_sup)  # returns only frequent item sets
print "\nFrequent lists: "
print t_id_list


Frequent lists: 
[['Bread', 'Butter', 'Jam', 'Coke', 'Milk'], [[1, 4, 5, 7, 8, 9], [1, 2, 3, 4, 6, 8, 9], [1, 8], [2, 4], [3, 5, 6, 7, 8, 9]]]


In [10]:
# this function generates the itemsets of the next level
# for example as shown above items sets Bread, Butter, Jam, Coke, Milk are generated
# therefore at the next level it will create following itemsets
# {Bread, Butter}, {Bread, Jam}, {Bread, Coke}, {Bread, Milk}, 
# {Butter, Jam}, {Butter, Coke}, {Butter, Milk}, {Jam, Coke},
# {Jam, Milk}, {Coke, Milk} 
def generate_next_level(t_id_list, level):  
    l = len(t_id_list[0])
    new_t_id_list = [[], []]

    item_list = []  #list containing sets of itemsets
    transaction_list = []   #list containing sets of transactions ids
    for i in range(0, l):
        items = t_id_list[0][i]
        items = items.split(",")
        sets = []
        for item in items:
            sets.append(item)
        item_list.append(set(sets)) #creates set of itemset
        transaction_list.append(set(t_id_list[1][i]))   #create set of transaction ids

    # now generate next level itemsets
    #suppose item_list is ["Butter", "Milk", "Bread"]
    for i in range(0, l):
        # pick one by one itemsets from the list
        #suppose item_set picked here is "Butter"
        item_set = item_list[i]
        t_ids = transaction_list[i]
        for j in range(i + 1, l):
            # try to generate the new itemsets by combining it with previously picked itemset
            #suppose item_set picked here is "Milk"
            #so it will combine Bread and Milk and generate new item set {Bread, Milk}
            item_set2 = item_list[j]
            
            #this handles the cases which are the likes of the following:
            # item_set = {Bread, Milk}
            # item_set2 = {Jam, Coke}
            # while generating itemsets at level 3, the length of the itemset must be 3
            # however, if we combine directly item_set with item_set2, it will generate the itemsets
            # with the length 4. Folloing code will run in such cases
            if len(list(item_set | item_set2)) > level:
                list1 = list(item_set)
                list2 = list(item_set2)
                m = level - 1
                t_ids2 = transaction_list[j]
                
                #in the above mentioned cases, it will be divided in two parts
                #in the first part, it will pick one item from the item_set one by one 
                # and combine it with item_set2
                # for example: from item_set = {Bread, Milk}, first pick Bread and combine it with 
                # item_set2. Then create Milk and combine it with item_set2
                # so, totak two sets will be generated: {Bread, Jam, Coke} and {Milk, Jam, Coke}
                for k in range(0, m):
                    item1 = set([list1[k]])
                    new_item_set = ",".join(item1 | item_set2)
                    
                    # proceed only if it's not generated earlier
                    if new_item_set not in new_t_id_list[0]:
                        t_ids1 = set(frequent_set[list1[k]])
                        new_t_ids = list(t_ids1 & t_ids2)
                        if new_t_ids:
                            new_t_id_list[0].append(new_item_set)
                            new_t_id_list[1].append(new_t_ids)
                            # print new_t_ids
                    # print "\n\n"

                # in the second part, it will pick item one by one from item_set2 and combine it with
                # item_set
                # for example: from item_set2 = {Jam, Coke}
                # Pick Jam and combine with item_set
                # pick Coke and combine with item_set
                # so generated itemsets: {Bread, Milk, Jam} and {Bread, Milk, Coke}
                # You can notice that all generated itemsets are of length 3
                for k in range(0, m):
                    item2 = set([list2[k]])

                    new_item_set = ",".join(item2 | item_set)
                    
                    # proceed only if it's not generated earlier
                    if new_item_set not in new_t_id_list[0]:
                        t_ids_2 = set(frequent_set[list2[k]])
                        new_t_ids = list(t_ids & t_ids_2)
                        if new_t_ids:
                            new_t_id_list[0].append(new_item_set)
                            new_t_id_list[1].append(new_t_ids)
                            # print new_t_ids
                    # print "\n\n"

            else:
                #the following line generates new itemsets using the union of the sets
                # for example: A = set(['Butter', 'Milk']), B = set(['Butter', 'Bread'])
                # then the output will be set(['Butter', 'Milk', 'Bread'])
                new_item_set = ",".join(item_set | item_set2)
                
                #proceed ahead only if the generated itemsets is unique (means not generated before)
                if new_item_set not in new_t_id_list[0]:
                    t_ids2 = transaction_list[j]
                    new_item_set = ",".join(item_set | item_set2)
                    
                    # transaction ids of the newly generated itemset can be generated easily by
                    # using "AND" operation of two sets
                    # for example:
                    # Butter = {1, 4, 5, 7, 9}
                    # Milk = {1, 5, 7, 8, 9}
                    # So, Butter, Milk = {1, 5, 7, 9}. All these transactions contains 
                    # Butter with Milk
                    new_t_ids = list(t_ids & t_ids2)
                    # there may be case, in which itmeset {Bread, Milk, Bread} are generated 
                    # but there is not a single transaction that includes all three items
                    # in such cases new_t_ids will be [] (empty). So, reject this kind of itemsets
                    if new_t_ids:
                        new_t_id_list[0].append(new_item_set)
                        new_t_id_list[1].append(new_t_ids)

    return new_t_id_list


In [11]:
# we already generated itemsets at level 1, so to generate at next level
# level is intialized with 2
level = 2

#this loop will generate item sets at the next level until possible
while t_id_list[0]:
    t_id_list = generate_next_level(t_id_list, level)   # generate next level

    print "\nLevel " + str(level) + ":"
    l = len(t_id_list[0])
    for i in range(0, l):                      # print all the generated item_sets at the next level
        print t_id_list[0][i], t_id_list[1][i]

    t_id_list = get_frequent_sets(t_id_list, min_sup)  # returns only frequent item sets
    print "\nFrequent lists: "
    print t_id_list
    print ""
    level += 1



Level 2:
Butter,Bread [8, 1, 4, 9]
Jam,Bread [8, 1]
Coke,Bread [4]
Milk,Bread [8, 9, 5, 7]
Butter,Jam [8, 1]
Butter,Coke [2, 4]
Butter,Milk [8, 9, 3, 6]
Jam,Milk [8]

Frequent lists: 
[['Butter,Bread', 'Jam,Bread', 'Milk,Bread', 'Butter,Jam', 'Butter,Coke', 'Butter,Milk'], [[8, 1, 4, 9], [8, 1], [8, 9, 5, 7], [8, 1], [2, 4], [8, 9, 3, 6]]]


Level 3:
Butter,Jam,Bread [8, 1]
Butter,Milk,Bread [8, 9]
Butter,Coke,Bread [4]
Jam,Milk,Bread [8]
Butter,Jam,Milk [8]

Frequent lists: 
[['Butter,Jam,Bread', 'Butter,Milk,Bread'], [[8, 1], [8, 9]]]


Level 4:
Butter,Jam,Milk,Bread [8]

Frequent lists: 
[[], []]



In [12]:
#this code will compute the support of the frequent items
for item_set in frequent_set.keys():
    # support is the number of times item_set occurs in the data
    support[item_set] = len(frequent_set[item_set])
    print "Support(" + item_set + ") is ", support[item_set]

Support(Butter) is  7
Support(Butter,Bread) is  4
Support(Coke) is  2
Support(Milk,Bread) is  4
Support(Butter,Milk,Bread) is  2
Support(Jam) is  2
Support(Butter,Jam) is  2
Support(Jam,Bread) is  2
Support(Butter,Milk) is  4
Support(Butter,Coke) is  2
Support(Butter,Jam,Bread) is  2
Support(Milk) is  6
Support(Bread) is  6


In [13]:
# this function calculate confidence of the itemsets
# Confidence is used to generate strong association rules
# if the {Butter, Bread, Milk} is the frequent set, then
# Confidence(Butter,Bread->Milk) basically is the probability 
# that user will buy Milk, given user has bought Butter and Bread
def calculate_confidence(item_set, sub_set):
    return float(support[item_set])/float(support[sub_set])


# for checking confidence, we need to get all the subset of the itemset,
# except the empty set and the itemset itself
# the following functin returns subsets of itemset
# for input (Butter, Milk, Jam) returns
# {Butter}, {Milk}, {Jam}, {Butter, Milk}, {Butter, Jam}, {Milk, Jam}
def get_subsets(item_set):
    item_set = item_set.split(",")
    set1 = []
    limit = len(item_set)
    for i in range(1, limit):
        for j in combinations(item_set, i):
            set1.append(j)
    return set1

In [14]:
# the following code will compute the confidence of each possible 
# combinations of itemsets
for item_set in frequent_set.keys():
    set1 = get_subsets(item_set)
    if set1:
        l = len(set1)
        limit = l
        l -= 1
        for i in range(0, limit):
            set1[i] = ",".join(set1[i])

        for i in range(0, limit):
            confidence = calculate_confidence(item_set, set1[i])
            if confidence >= min_confidence:
                confident_set[set1[i]+"->"+set1[l-i]] = confidence
for item in confident_set.keys():
    print "confidence(" + item + ") is", confident_set[item]

confidence(Jam->Butter) is 1.0
confidence(Milk->Bread) is 0.666666666667
confidence(Bread->Butter) is 0.666666666667
confidence(Jam->Bread) is 1.0
confidence(Butter,Jam->Bread) is 1.0
confidence(Coke->Butter) is 1.0
confidence(Jam,Bread->Butter) is 1.0
confidence(Jam->Butter,Bread) is 1.0
confidence(Bread->Milk) is 0.666666666667
confidence(Milk->Butter) is 0.666666666667
